# Flare advocates analysis notebook
## Prerequisities: in order to run this notebook you need the AGILE data.

In [ ]:
from agilepy.api.AGAnalysis import AGAnalysis

In [ ]:
%matplotlib notebook

## Analysis name

In [ ]:
user = "bulgarelli"
sourceName = "sourceXXX"
analysisname = user+"_"+sourceName

In [ ]:
print("Analysis:",analysisname)

## Configuration 

In [ ]:
tmin = 58884.0 # 2020-02-05 00:00:00.000
tmax = 58886.0 # 2020-02-07 00:00:00.000
timetype = "MJD" # "MJD" OR "TT"
glon = 79.8
glat = 0.7
outDir = f"$HOME/agilepy_analysis/{analysisname}/output"

In [ ]:
config = """
input:
  evtfile: /AGILE_PROC3/FM3.119_ASDC2/INDEX/EVT.index
  logfile: /AGILE_PROC3/DATA_ASDC2/INDEX/LOG.log.index

output:
  outdir: %s
  filenameprefix: %s_product
  logfilenameprefix: %s_log
  verboselvl: 1

selection:
  tmin: %f
  tmax: %f
  timetype: %s
  glon: %f
  glat: %f
  fovradmax: 60
  albedorad: 80
  proj: ARC
  
maps:
  mapsize: 40
  spectralindex: 2.1
  timestep: 160
  binsize: 0.25
  energybins:
    - 100, 10000
  fovbinnumber: 1
  

model:
  modelfile: null
  galmode: 1
  isomode: 1
  galcoeff: null
  isocoeff: null
  emin_sources: 100
  emax_sources: 10000


mle:
  ranal: 10
  ulcl: 2
  loccl: 95
  expratioevaluation: yes
  expratio_minthr: 0
  expratio_maxthr: 15
  expratio_size: 10  
  
"""%(outDir, analysisname, analysisname, tmin, tmax, timetype, glon, glat)

with open("./agilepy_conf.yaml", "w") as cf:
    cf.write(config)

## Obtaining the AGAnalysis object

In [ ]:
ag = AGAnalysis("./agilepy_conf.yaml")

## Sources hypothesis

In [ ]:
sources = ag.loadSourcesFromCatalog("2AGL", rangeDist = (0, 10))

In [ ]:
for s in sources:
    print(s)

## Adding a source at runtime

In [ ]:
newSourceDict = {
    "glon" : 79.3,
    "glat": 0.554,
    "spectrumType" : "PowerLaw",
    "flux": 1.69737e-07,
    "index": 2.1
}

s = ag.addSource("CYGX3", newSourceDict)

print(s)

## Deleting sources
Selection params = [name, dist, flux, sqrtts]

In [ ]:
deletedSources = ag.deleteSources('dist >= 3')

## Selecting sources

In [ ]:
sources = ag.selectSources("flux > 0")

## Free a source's parameter
Freeable params = [flux, index, index1, index2, cutoffEnergy, pivotEnergy, curvature, index2]

In [ ]:
affectedSources = ag.freeSources("flux > 0", "cutoffEnergy", True)

In this example, only one source is affected: the one with spectrum type "PLExpCutoff" (it is the only spectrum type that has the "cutoffEnergy" parameter)

In [ ]:
for source in affectedSources:
    print(source)

## Analysis

In [ ]:
ag.generateMaps()

In [ ]:
ag.displayCtsSkyMaps(smooth=3, catalogRegions="2AGL", catalogRegionsColor="green")

In [ ]:
ag.displayExpSkyMaps()

In [ ]:
ag.displayGasSkyMaps()

In [ ]:
ag.calcBkg("CYGX3", galcoeff = None, pastTimeWindow = 14)

In [ ]:
ag.mle()

In [ ]:
ag.selectSources("sqrtTS > 0")